In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("../results/enriched_routes/beynes_routes_enriched.csv", quotechar='"')



In [3]:

# Cechy ciągłe do normalizacji
continuous_features = [
    'free_flow_time',
    # 'total_length',
    'mean_speed',
    'speed_std',
    'speed_range',
    # 'lane_changes_per_km',
    # 'priority_changes_per_km',
    # 'yield_priority_changes_per_km',
    'traffic_lights_per_km',
    # 'bearing_std',
    # 'turns_per_km',
    'left_yield_turns_per_km',
    # 'mean_circuity',
    # 'edge_length_std',
    # 'edges_per_km'
]

# Procentowe cechy w skali 0–1 (nie wymagają normalizacji)
percentage_features = [
    # 'pct_high_speed',
    # 'pct_motorway',
    # 'pct_trunk',
    # 'pct_primary',
    # 'pct_secondary',
    # 'pct_tertiary',
    # 'pct_unclassified',
    # 'pct_residential'
]

# Wszystkie cechy używane w modelu
features = continuous_features + percentage_features



In [4]:
import numpy as np
# 3. Sprawdzenie braków danych
print("Braki danych w kolumnach:")
print(df[features].isna().sum())
print(df[features].isna().sum()[lambda x: x > 0])
df[features] = df[features].fillna(df[features].median())
print(df[features].isna().sum()[lambda x: x > 0])
# print(df['pct_unpaved'].value_counts())
# # print(df['bridges_per_km'].value_counts())
# columns_to_check = ['tunnels_per_km', 'bridges_per_km', 'pct_motorway', 'pct_trunk', 'pct_lit', 'pct_unpaved']

# # Liczenie zer i innych wartości
# for col in columns_to_check:
#     zeros = (df[col] == 0).sum()
#     non_zeros = (df[col] != 0).sum()
#     print(f"{col}: zeros = {zeros}, non-zeros = {non_zeros}")


Braki danych w kolumnach:
free_flow_time             0
mean_speed                 0
speed_std                  0
speed_range                0
traffic_lights_per_km      0
left_yield_turns_per_km    0
dtype: int64
Series([], dtype: int64)
Series([], dtype: int64)


In [5]:

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[continuous_features] = scaler.fit_transform(df[continuous_features])
# weighted mean_speed
weight = 10.0
df_scaled['mean_speed'] = df_scaled['mean_speed'] * weight

X = df_scaled[features].values

print("Dane gotowe do clusteringu:", X.shape)


Dane gotowe do clusteringu: (4048, 6)


In [6]:
# Liczymy ile ścieżek ma te same origin i destination
df['num_paths_same_od'] = df.groupby(['origins', 'destinations'])['path'].transform('count')
print(df[['origins', 'destinations', 'num_paths_same_od']].head(10))

# Nadanie unikalnego numeru ścieżki dla każdej pary origin-destination
df['route_id'] = df.groupby(['origins', 'destinations']).ngroup()
print(df[['origins', 'destinations', 'route_id']].head(10))

    origins destinations  num_paths_same_od
0  57680159   41273128#1                 20
1  57680159   41273128#1                 20
2  57680159   41273128#1                 20
3  57680159   41273128#1                 20
4  57680159   41273128#1                 20
5  57680159   41273128#1                 20
6  57680159   41273128#1                 20
7  57680159   41273128#1                 20
8  57680159   41273128#1                 20
9  57680159   41273128#1                 20
    origins destinations  route_id
0  57680159   41273128#1       203
1  57680159   41273128#1       203
2  57680159   41273128#1       203
3  57680159   41273128#1       203
4  57680159   41273128#1       203
5  57680159   41273128#1       203
6  57680159   41273128#1       203
7  57680159   41273128#1       203
8  57680159   41273128#1       203
9  57680159   41273128#1       203


In [7]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from collections import Counter, defaultdict

def constrained_kmeans_with_route_penalty(X, route_ids, n_clusters=3, 
                                          lambda_penalty=1.0, 
                                          f_type='pairs',
                                          init=None,
                                          max_iter=100,
                                          tol=1e-4,
                                          random_state=42,
                                          n_init=5):
    """
    Prosty greedy K-means z dodatkową karą za powtarzające się route_id w klastrze.

    Args:
        X: ndarray (n_samples, n_features)
        route_ids: iterable z kategoriami (np. pandas Series) długości n_samples
        n_clusters: liczba klastrów
        lambda_penalty: waga kary
        f_type: 'pairs' -> f(n)=n*(n-1)/2, 'linear' -> f(n)=n-1 (dla n>=1)
        init: opcjonalne init centroids ndarray (K, d) lub None -> użyj KMeans do inicjalizacji
        max_iter: maksymalna liczba iteracji
        tol: tolerancja zmiany centroidów
        n_init: ile restartów (wybieramy najlepszy wg funkcji celu J)
    Returns:
        best_labels, best_centers, diagnostics (dict)
    """
    n, d = X.shape
    route_ids = np.asarray(route_ids)
    unique_routes = np.unique(route_ids)

    def penalty_function(count):
        if count <= 1:
            return 0.0
        if f_type == 'pairs':
            return count * (count - 1) / 2.0
        elif f_type == 'linear':
            return count - 1
        else:
            raise ValueError("Unknown f_type")

    best_J = np.inf
    best_result = None

    rng = np.random.RandomState(random_state)

    for init_run in range(n_init):
        # Inicjalizacja centroidów
        if init is None:
            km = KMeans(n_clusters=n_clusters, n_init=1, random_state=rng.randint(0,10000))
            km.fit(X)
            centers = km.cluster_centers_.copy()
        else:
            centers = init.copy()

        labels = np.full(n, -1, dtype=int)

        for it in range(max_iter):
            changed = False

            # Przygotuj liczniki route w klastrach (aktualne przed przypisaniem)
            cluster_route_counts = [Counter() for _ in range(n_clusters)]
            # jeśli mamy stare etykiety, wypełniamy liczniki
            for i, lab in enumerate(labels):
                if lab != -1:
                    cluster_route_counts[lab][route_ids[i]] += 1

            # Krok przypisania (greedy): dla każdego punktu obliczamy koszt dla każdego klastra
            # uwzględniamy obecną liczbę route w klastrze (przed dodaniem punktu)
            new_labels = labels.copy()
            for i in range(n):
                xi = X[i]
                ri = route_ids[i]
                best_k = None
                best_cost = np.inf
                for k in range(n_clusters):
                    # squared distance
                    dist2 = np.sum((xi - centers[k])**2)
                    # kara po przypisaniu: policz ile by było w klastrze po dodaniu tego punktu
                    cnt_after = cluster_route_counts[k][ri] + (1 if labels[i] != k else 0)
                    # możemy policzyć incremental penalty: f(cnt_after) - f(cnt_before)
                    cnt_before = cluster_route_counts[k][ri] - (1 if labels[i] == k else 0)
                    # ale prostsze: użyj cnt_after jako g(k,ri) (liczba duplikatów po dodaniu)
                    if f_type == 'pairs':
                        pen_before = penalty_function(cluster_route_counts[k][ri])
                        pen_after = penalty_function(cluster_route_counts[k][ri] + 1)
                        incremental_penalty = pen_after - pen_before
                    else:
                        # linear: each extra adds 1 (if there was 0 -> 0, next ->1, etc.)
                        incremental_penalty = 1.0 if cluster_route_counts[k][ri] >= 1 else 0.0
                    cost = dist2 + lambda_penalty * incremental_penalty
                    # jeśli punkt już był w tym samym klastrze, incremental_penalty powinno być 0 (bo nie zmieniamy)
                    if labels[i] == k:
                        # przypisanie pozostaje, więc incremental_penalty=0 i dist2 liczy się tak samo
                        pass
                    if cost < best_cost:
                        best_cost = cost
                        best_k = k
                # przypisz
                if best_k is None:
                    best_k = rng.randint(0, n_clusters)
                if new_labels[i] != best_k:
                    # zdejmij z poprzedniego (jeśli było)
                    if new_labels[i] != -1:
                        cluster_route_counts[new_labels[i]][route_ids[i]] -= 1
                    # dodaj do nowego
                    cluster_route_counts[best_k][route_ids[i]] += 1
                    new_labels[i] = best_k
                    changed = True

            labels = new_labels

            # Krok aktualizacji centroidów
            new_centers = np.zeros_like(centers)
            counts = np.zeros(n_clusters, dtype=int)
            for k in range(n_clusters):
                inds = np.where(labels == k)[0]
                if inds.size > 0:
                    new_centers[k] = X[inds].mean(axis=0)
                    counts[k] = inds.size
                else:
                    # pusty klaster -> re-inicjalizuj centroid losowo
                    new_centers[k] = X[rng.randint(0, n), :]

            center_shift = np.sqrt(((centers - new_centers)**2).sum(axis=1)).max()
            centers = new_centers
            if not changed or center_shift < tol:
                break

        # Po zakończeniu iteracji – oblicz ostateczny J (całkowity koszt)
        inertia = np.sum((X - centers[labels])**2)
        # kara: dla każdego klastra i route policz f(n)
        total_penalty = 0.0
        cluster_route_counts = [Counter() for _ in range(n_clusters)]
        for i, lab in enumerate(labels):
            cluster_route_counts[lab][route_ids[i]] += 1
        for k in range(n_clusters):
            for r, cnt in cluster_route_counts[k].items():
                total_penalty += penalty_function(cnt)
        J = inertia + lambda_penalty * total_penalty

        if J < best_J:
            best_J = J
            best_result = dict(labels=labels.copy(), centers=centers.copy(), J=J,
                               inertia=inertia, total_penalty=total_penalty, it=it, init_run=init_run)

    return best_result




In [11]:
# ---------- Przykładowe użycie ----------
# Zakładamy, że masz df (pandas) i X przygotowane tak jak wcześniej, oraz df['route_id'].
# X: numpy array [n_samples, n_features]
# route_ids: df['route_id']

result = constrained_kmeans_with_route_penalty(X, df['route_id'], n_clusters=10, lambda_penalty=1.0, f_type='pairs')
df['cluster'] = result['labels']

print(df['cluster'].value_counts())
df.to_csv("../results/clustered_routes/beynes_routes_clustered.csv", index=False)


cluster
6    700
0    661
1    527
9    512
3    508
8    507
2    330
5    144
7    122
4     37
Name: count, dtype: int64


In [12]:

global_speed = df['mean_speed'].mean()
print(global_speed)
# Średnia prędkość w każdym klastrze
cluster_speed = df.groupby('cluster')['mean_speed'].mean()
print(cluster_speed)

# Który klaster najszybszy
fastest_cluster = cluster_speed.idxmax()
print(f"Najszybszy klaster to: {fastest_cluster}")

35.37449526412878
cluster
0    29.961404
1    38.669715
2    43.987356
3    33.082707
4    56.075960
5    25.350278
6    30.314826
7    49.207217
8    35.065246
9    40.944723
Name: mean_speed, dtype: float64
Najszybszy klaster to: 4


In [14]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

counts = df.groupby(["route_id", "cluster"]).size().reset_index(name="count")
print(counts)
dup = counts[counts["count"] < 1]
print(dup)
print(df['route_id'].unique().shape)



     route_id  cluster  count
0           0        1     15
1           0        8      1
2           0        9      4
3           1        1     11
4           1        2      1
5           1        9      8
6           2        1     11
7           2        2      1
8           2        8      1
9           2        9      7
10          3        0      8
11          3        3      3
12          3        6      9
13          4        0     10
14          4        6     10
15          5        1     10
16          5        2      1
17          5        9      9
18          6        1     16
19          6        9      4
20          7        2     10
21          7        7      5
22          7        9      5
23          8        1      1
24          8        3      1
25          8        8      8
26          9        0     10
27          9        6     10
28         10        1     12
29         10        8      1
30         10        9      7
31         11        2     12
32        